In [ ]:
import sympy as sp
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from IPython.display import Markdown as md

# Definer variable her
R = sp.symbols("R")
variables = [R]

V_kugle = (4/3) * sp.pi * R**3

# En funktion der returnerer en liste med de partielle afledede af et udtryk
def partial_derivatives(expr):
    return [(sp.diff(expr, var), var) for var in variables]

# Brug ophobningsloven til at finde udtrykket for usikkerheden på g for et givent udtryk
# Tager et udtryk, en liste med usikkerheder på parametrene og en liste med parametre fra et fit som input
def prop_uncert(expr, uncertainties):
    partials = partial_derivatives(expr)
    expr = sp.sqrt(sum([(partials[i][0] * uncertainties[i])**2 for i in range(len(partials))]))
    return expr

prop_uncert(V_kugle, [0.001])


